In [13]:
import pandas as pd

In [14]:
new_market = pd.read_csv('./new_market.csv')

In [15]:
new_market['portfolio_value'] = None

In [16]:
initial_investment = 10000

num_stocks = 485

a = []

# 각 주식 종목의 일별 주가 변화율 계산
for column in new_market.columns[1:]:  # 첫 번째 열(날짜)을 제외하고 계산
    daily_return = (new_market[column] - new_market[column].shift(1)) / new_market[column].shift(1)
    investment_return = daily_return * (initial_investment / num_stocks)
    a.append(investment_return)

# NaN 값 처리 및 포트폴리오 가치 계산
portfolio_values = pd.DataFrame(a).sum().fillna(0)

# 첫 번째 행의 'portfolio_value'는 초기 투자 금액으로 설정
portfolio_values.iloc[0] = initial_investment

# 이후 행들에 대해 누적 합산을 사용하여 포트폴리오 가치 계산
for i in range(1, len(portfolio_values)):
    portfolio_values.iloc[i] += portfolio_values.iloc[i - 1]

# 결과 확인
portfolio_values.head()  # 계산된 portfolio_values의 첫 5개 값을 확인합니다.

0    10000.000000
1     9927.705008
2     9910.896654
3    10034.582108
4    10059.932739
dtype: float64

In [17]:
new_market['portfolio_value'] = portfolio_values.values

In [ ]:
from datetime import datetime, timedelta
import calendar

def find_next_valid_date(df, start_date):
    """
    주어진 날짜로부터 3개월 후의 유효한 날짜를 찾는 함수.
    만약 3개월 후의 그 날짜가 없으면, 이후의 유효한 날짜를 찾는다.
    """
    # 시작 날짜를 datetime 객체로 변환
    current_date = datetime.strptime(start_date, "%Y-%m-%d")

    # 3개월 뒤 날짜 계산
    month = current_date.month - 1 + 3
    year = current_date.year + month // 12
    month = month % 12 + 1
    day = 1

    # 유효한 날짜 찾기
    while True:
        try:
            next_date = datetime(year, month, day).strftime("%Y-%m-%d")
            if next_date in df['pricingDate'].values:
                return next_date
            else:
                day += 1
        except ValueError:
            # 해당 월에 day가 없는 경우 (예: 2월 30일)
            day = 1
            month += 1

